In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import random

In [ ]:
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_GRAYSCALE) #이미지를 흑백이미지로 읽음
plt.imshow(gray_lena_img, cmap='gray') #흑백의 원본 사진 출력 512x512
plt.title('gray lena img')
plt.show()

gray_pepper = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/pepper.bmp', cv2.IMREAD_GRAYSCALE) #이미지를 흑백이미지로 읽음
plt.imshow(gray_pepper, cmap='gray') #흑백의 원본 사진 출력
plt.title('gray_pepper')
plt.show()

gray_landscape_forest_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Forest Path.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_landscape_forest_path, cmap='gray')
plt.title('gray_landscape_forest_path')
plt.show()

gray_landscape_rocky_stream = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Rocky Stream.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_landscape_rocky_stream, cmap='gray')
plt.title('gray_landscape_rocky_stream')
plt.show()

gray_landscape_snowy_mountains = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Snowy Mountains.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_landscape_snowy_mountains, cmap='gray')
plt.title('gray_landscape_snowy_mountains')
plt.show()

gray_landscape_springtime_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Springtime Path.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_landscape_springtime_path, cmap='gray')
plt.title('gray_landscape_springtime_path')
plt.show()

gray_landscape_sunriese_lake = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Sunrise Lake.png',cv2.IMREAD_GRAYSCALE)
plt.imshow(gray_landscape_sunriese_lake, cmap='gray')
plt.title('gray_landscape_sunriese_lake')
plt.show()


In [ ]:
# 1번
def gaussian_noise_img(img, sigma): # (0, sigma^2)인 가우시안 노이즈를 이미지에 추가
    M,N=img.shape
    gaussian_noise=np.random.normal(0,sigma,(M,N)) #(0, sigma^2)인 가우시안 노이즈를 이미지의 크기만큼 생성
    gaussian_noise_img=img+gaussian_noise #원본 영상에 가우시안 노이즈 추가
    gaussian_noise_img = np.clip(gaussian_noise_img, 0, 255)
    return gaussian_noise_img.astype(np.uint8) #노이즈가 추가된 이미지 반환

def add_impulse_noise(img):
    M,N=img.shape
    impulse_img=np.copy(img)

    number_of_pixels = random.randint(int(M*N/100), int(M*N/10)) #임의로 이미지에서 픽셀 선택. 임의로 이미지의 가로,세로 범위 설정
    for i in range(number_of_pixels):
        x = random.randint(0, N- 1) # 무작위로 x 좌표 선택
        y = random.randint(0, M- 1) # 무작위로 y 좌표 선택
        impulse_img[x][y] = 255 #흑백 이미지를 사용하므로 무작위로 선택한 픽셀값을 눈에 잘 띄는 흰색으로 바꿔준다

    return impulse_img

def psnr_func(image1,image2): # 2개의 이미지의 크기기 같아야 됨
    mse=np.mean((image1-image2)**2) #mse 계산식
    psnr=20*math.log10(255/math.sqrt(mse)) #psnr 계산식
    return psnr

#median - 필터가 지나가면서 필터의 중간값을 새로운 픽셀값으로 리턴한다
def median_filter(img,k_size): #필터링할 이미지와 커널의 사이즈를 입력값으로 넘겨준다
  M, N = img.shape

  img_pad = np.zeros((M+2, N+2)) # medan 3x3 으로 진행
  img_pad[1:1+M, 1:1+N] = img.copy() #패딩된 행렬에 원본 이미지를 옮겨 붙인다
  temp_img_pad = img_pad.copy()

  img_out = np.zeros((M, N)) # 노이즈를 없앤 이미지를 담을 배열

  for i in range(M):
    for j in range(N):
      img_pad[1+i, 1+j] = np.median(temp_img_pad[i:i+k_size, j:j+k_size]) # median operation--------수정 필요

  img_out = img_pad[1:1+M, 1:1+N] #패딩 제거한 이미지
  img_out=np.sqrt(img_out**2)
  img_out *=255/img_out.max()

  return img_out #패딩한 행렬에 median 필터링 연산을 진행한 뒤 패딩을 제거해 준 이미지를 출력

#bilateral - 에지가 아닌 부분에서만 블러링 효과. 가우시안 보다 나은 성능
def bilateral_filter(image, sigma_s, sigma_r, window_size):
    # sigma_s: 공간적 가중치 범위 조절. 크면 더 넓은 영역 평활화
    # sigma_r: 강도 차이 민감도 저절. 크면 엣지 보존 약화됨
    half_window = window_size // 2 # 중심 픽셀을 기준으로 확장하는 정도
    filtered_image = np.zeros_like(image)
    # 이미지의 경계를 벗어나지 않도록 순회
    for y in range(half_window, image.shape[0] - half_window):
        for x in range(half_window, image.shape[1] - half_window):
            center_intensity = image[y, x] # 중심 픽셀 강도(픽셀값)
            # 공간가중치 계산(Gs)
            spatial_weights = np.exp(-((np.arange(-half_window, half_window + 1) / sigma_s) ** 2))
            # 강도가중치 계산(Gr)-강도차이가 큰 픽셀(엣지)에 낮은 가중치 부여함으로 엣지 유지
            intensity_weights = np.exp(-((image[y - half_window:y + half_window + 1, x - half_window:x + half_window + 1] - center_intensity) ** 2) / (2 * sigma_r ** 2))
            # 가중치 결합-공간적으로 가깝고 강도가 유사한 픽셀만 평균내어 노이즈 제거
            weights = spatial_weights * intensity_weights
            filtered_image[y, x] = np.sum(weights * image[y - half_window:y + half_window + 1, x - half_window:x + half_window + 1]) / np.sum(weights)

    return filtered_image

# 가우시안 필터---블러링 효과
def gaussian_filter(k,sig): # (필터사이즈, 표준편차)
    ax = np.linspace(-(k//2), k//2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x**2 + y**2) / (2 * sig**2)) # k x k 크기의 커널 생성
    return kernel / np.sum(kernel) # 정규화. 전체 필터의 요소들의 합은 1

# 가우시안 필터
gaussian_filter_mask=gaussian_filter(5,1)

# 컨볼루션 진행 함수
def convolution(image, filter):
  image_height, image_width = image.shape
  filter_height, filter_width = filter.shape
  padding = filter_height // 2
  filtered_image = np.zeros_like(image) #새로운 복사 이미지

  #반복문을 수행하면서 패딩&컨볼루션 진행
  for y in range(padding, image_height - padding):
    for x in range(padding, image_width - padding):
      image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
      filtered_image[y, x] = np.sum(image_patch * filter)

  return filtered_image


In [ ]:
# 1) Generate the following noises, and add them to the ground-truth images. Compute PSNR for noisy images.
# a) Gaussian noise with (mean, var) = (0, 𝜎2)
# b) Impulse noise

g_noise_img=gaussian_noise_img(gray_lena_img,50) #시그마의 값에 따라 이미지 블러링 정도가 다르다
impulse_noise_img=add_impulse_noise(gray_lena_img)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(gray_lena_img,cmap='gray'); plt.title('original img')
plt.subplot(1,3,2); plt.imshow(g_noise_img,cmap='gray'); plt.title('gaussian noise img sigma=50')
plt.subplot(1,3,3); plt.imshow(impulse_noise_img,cmap='gray'); plt.title('impulse noise img')
plt.show()


In [ ]:
#2) Denoise the generated noisy images. Compute PSNR for denoised images.
#bilateral, median, gaussian 사용해서 노이즈 제거 효과 비교

gaussian_filtered_img1=convolution(g_noise_img,gaussian_filter_mask)
gaussian_filtered_img2=convolution(impulse_noise_img,gaussian_filter_mask)

plt.figure(figsize=(10,10))
plt.subplot(2,2,1); plt.imshow(g_noise_img,cmap='gray'); plt.title('gaussian noise img sigma=50')
plt.subplot(2,2,2); plt.imshow(gaussian_filtered_img1,cmap='gray'); plt.title('gaussian filtered image - gaussian noise version')
plt.subplot(2,2,3); plt.imshow(impulse_noise_img,cmap='gray'); plt.title('impulse noise img')
plt.subplot(2,2,4); plt.imshow(gaussian_filtered_img2,cmap='gray'); plt.title('gaussian filtered image - impulse noise version')
plt.show()


#gaussian filter의 가우시안 노이즈 이미지의 mse와 psnr 계산
gaussian_psnr = psnr_func(gaussian_filtered_img1, gray_lena_img)
print("gaussian noise psnr : ", gaussian_psnr)

#gaussian filter의 임펄스 노이즈 이미지의 mse와 psnr 계산
gaussian_psnr = psnr_func(gaussian_filtered_img2, gray_lena_img)
print("impulse noise psnr : ", gaussian_psnr)


In [ ]:
# median filter 수행. 필터가 지나가면서 필터의 중간값을 새로운 픽셀값으로 리턴한다
median_img1=median_filter(g_noise_img,3) #가우시안 노이즈가 추가된 이미지 필터링 결과
median_img2=median_filter(impulse_noise_img,3) #임펄스 노이즈가 추가된 이미지 필터링 결과

plt.figure(figsize=(10,10))
plt.subplot(2,2,1); plt.imshow(g_noise_img,cmap='gray'); plt.title('gaussian noise img sigma=50')
plt.subplot(2,2,2); plt.imshow(median_img1,cmap='gray'); plt.title('median filtered image - gaussian noise version')
plt.subplot(2,2,3); plt.imshow(impulse_noise_img,cmap='gray'); plt.title('impulse noise img')
plt.subplot(2,2,4); plt.imshow(median_img2,cmap='gray'); plt.title('median filtered image - impulse noise version')
plt.show()

#median filter의 가우시안 노이즈 이미지의 mse와 psnr 계산
median_psnr = psnr_func(median_img1, gray_lena_img)
print("gaussian noise psnr : ", median_psnr)

#median filter의 임펄스 노이즈 이미지의 mse와 psnr 계산
median_psnr = psnr_func(median_img2, gray_lena_img)
print("impulse noise psnr : ", median_psnr)


In [ ]:
# hybrid median filter - median을 5x5로 진행
hybrid_median_img1=median_filter(g_noise_img,5) #가우시안 노이즈가 추가된 이미지 필터링 결과
hybrid_median_img2=median_filter(impulse_noise_img,5) #임펄스 노이즈가 추가된 이미지 필터링 결과

plt.figure(figsize=(10,10))
plt.subplot(2,2,1); plt.imshow(g_noise_img,cmap='gray'); plt.title('gaussian noise img sigma=50')
plt.subplot(2,2,2); plt.imshow(hybrid_median_img1,cmap='gray'); plt.title('hybrid median filtered image - gaussian noise version')
plt.subplot(2,2,3); plt.imshow(impulse_noise_img,cmap='gray'); plt.title('impulse noise img')
plt.subplot(2,2,4); plt.imshow(hybrid_median_img2,cmap='gray'); plt.title('hybrid median filtered image - impulse noise version')
plt.show()

#median filter의 가우시안 노이즈 이미지의 mse와 psnr 계산
hybrid_median_psnr = psnr_func(hybrid_median_img1, gray_lena_img)
print("gaussian noise psnr : ", hybrid_median_psnr)

#median filter의 임펄스 노이즈 이미지의 mse와 psnr 계산
hybrid_median_psnr = psnr_func(hybrid_median_img2, gray_lena_img)
print("impulse noise psnr : ", hybrid_median_psnr)


In [ ]:
# bilater filter - 에지가 아닌 부분에서만 블러링 효과. 가우시안 보다 나은 성능,  가우시안 필터링을 양쪽 방향으로 2번 진행
bilateral_img1=bilateral_filter(g_noise_img,1,1,3)
bilateral_img2=bilateral_filter(impulse_noise_img,1,1,3)

plt.figure(figsize=(10,10))
plt.subplot(2,2,1); plt.imshow(g_noise_img,cmap='gray'); plt.title('gaussian noise img sigma=50')
plt.subplot(2,2,2); plt.imshow(bilateral_img1,cmap='gray'); plt.title('bilatered image - gaussian noise version')
plt.subplot(2,2,3); plt.imshow(impulse_noise_img, cmap='gray') ; plt.title('impulse noise img')
plt.subplot(2,2,4); plt.imshow(bilateral_img2,cmap='gray'); plt.title('bilatered image - impulse noise version')
plt.show()

# bilater 필터의 가우시안 노이즈 이미지 psnr 계산
bilateral_psnr = psnr_func(bilateral_img1, gray_lena_img)
print("gaussian noise psnr : ", bilateral_psnr)
# bilater 필터의 임펄스 노이즈 이미지 psnr 계산
bilateral_psnr = psnr_func(bilateral_img2, gray_lena_img)
print("impulse noise psnr : ", bilateral_psnr)



In [ ]:
# bilinear 보간 흑백 버전
def bilinear_interpolation(img, target_shape):
    h, w = img.shape
    new_h, new_w = target_shape

    output = np.zeros(target_shape, dtype=img.dtype)

    scale_x = w / new_w
    scale_y = h / new_h

    for i in range(new_h):
        for j in range(new_w):
            x = j * scale_x
            y = i * scale_y
            x0 = int(x)
            y0 = int(y)

            x1 = min(x0 + 1, w - 1)
            y1 = min(y0 + 1, h - 1)

            dx = x - x0
            dy = y - y0

            output[i, j] = (1 - dx) * (1 - dy) * img[y0, x0] + \
                           dx * (1 - dy) * img[y0, x1] + \
                           (1 - dx) * dy * img[y1, x0] + \
                           dx * dy * img[y1, x1]

    return output


In [ ]:
# bicubic 보간 흑백 버전
def bicubic_interpolation(img, target_shape):
    h, w = img.shape
    new_h, new_w = target_shape
    output = np.zeros(target_shape, dtype=img.dtype)
    scale_x = w / new_w
    scale_y = h / new_h
    for i in range(new_h):
        for j in range(new_w):
            x = j * scale_x
            y = i * scale_y
            x0 = int(x)
            y0 = int(y)
            x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
            y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
            dx = x - x0
            dy = y - y0
            def cubic(t):
                t = np.abs(t)
                if t <= 1:
                    return 1 - 2*t**2 + t**3
                elif t <= 2:
                    return -t**3 + 5*t**2 - 8*t + 4
                return 0
            value = 0
            for m in range(4):
                for n in range(4):
                    weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                    value += weight * img[y_indices[n], x_indices[m]]
            output[i, j] = np.clip(value, 0, 255)
    return output


In [ ]:
# 최근접 보간 흑백 버전
# 코드가 간결하고 속도가 빠르다는 장점
# 이미지가 계산 현상을 보인다는 단점
def NN_interpolation(img, target_shape):
    h, w = img.shape
    new_h, new_w = target_shape

    output = np.zeros(target_shape, dtype=img.dtype)
    # 출력 이미지의 크기에 대한 입력 이미지의 비율
    scale_x = w / new_w
    scale_y = h / new_h

    for i in range(new_h): # i: y축
        for j in range(new_w): # j: x축
        # 좌표를 구한뒤 int로 반올림하여 가장 가까운 픽셀 선택
            x = min(int(j * scale_x), w -1) # min(,-1)를 통해 경계를 벗어나지 않도록 처
            y = min(int(i * scale_y), h -1)
            output[i, j] = img[y, x]

    return output



In [ ]:
# B-spline 보간 흑백 버전
def b_spline_interpolation(img, target_shape):
    #3차 B-spline 커널 함수: 거리 u에 대한 가중치 계산
    def bspline_kernel(u):
        u = abs(u)
        if u < 1:
            return (1/6) * (3 * u**3 - 6 * u**2 + 4)
        elif u < 2:
            return (1/6) * (-u**3 + 6 * u**2 - 12 * u + 8)
        return 0

    h, w = img.shape
    new_h, new_w = target_shape
    output = np.zeros(target_shape, dtype=img.dtype)
    scale_x = w / new_w
    scale_y = h / new_h
    for i in range(new_h):
        for j in range(new_w):
            # 출력 픽셀 (i, j)에 대응하는 입력 이미지의 좌표
            x = j * scale_x
            y = i * scale_y

             # 정수 좌표와 소수 부분 계산
            x_int = int(x)
            y_int = int(y)
            x_frac = x - x_int
            y_frac = y - y_int

            # 4x4 주변 픽셀에 대해 B-spline 가중치 계산
            value = 0.0
            for m in range(-1, 3):  # y 방향: -1, 0, 1, 2
                for n in range(-1, 3):  # x 방향: -1, 0, 1, 2
                    # 입력 이미지의 좌표
                    x_idx = min(max(x_int + n, 0), w - 1)  # 경계 처리
                    y_idx = min(max(y_int + m, 0), h - 1)  # 경계 처리

                    # B-spline 커널 가중치 계산
                    weight_x = bspline_kernel(n - x_frac)
                    weight_y = bspline_kernel(m - y_frac)
                    weight = weight_x * weight_y

                    # 가중치 적용하여 픽셀 값 누적
                    value += img[y_idx, x_idx] * weight

            # 출력 픽셀 값 저장 (정수로 반올림)
            output[i, j] = np.clip(round(value), 0, 255).astype(img.dtype)

    return output


In [ ]:
# psnr, mse 계산 함수
def psnr_func(image1,image2): # 2개의 이미지의 크기기 같아야 됨
    mse=np.mean((image1-image2)**2) #mse 계산식
    psnr=20*math.log10(255/math.sqrt(mse)) #psnr 계산식
    return psnr

# 가우시안 필터---블러링 효과
def gaussian_filter(k,sig): # (필터사이즈, 표준편차)
    ax = np.linspace(-(k//2), k//2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x**2 + y**2) / (2 * sig**2)) # k x k 크기의 커널 생성
    return kernel / np.sum(kernel) # 정규화. 전체 필터의 요소들의 합은 1

# 가우시안 필터
gaussian_filter_mask=gaussian_filter(5,1)

# 컨볼루션 진행 함수
def convolution(image, filter):
  image_height, image_width = image.shape
  filter_height, filter_width = filter.shape
  padding = filter_height // 2
  filtered_image = np.zeros_like(image) #새로운 복사 이미지

  #반복문을 수행하면서 패딩&컨볼루션 진행
  for y in range(padding, image_height - padding):
    for x in range(padding, image_width - padding):
      image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
      filtered_image[y, x] = np.sum(image_patch * filter)

  return filtered_image

# 가우시안 피라미드 생성 함수
def gaussian_pyramid(image, scale_num, levels): # scale_num: 0~1
    pyramid_img = [image]
    downsample_num = int(1 // scale_num)
    for _ in range(levels - 1): # 반복 변수를 직접 사용하지 않을 때 관습적 표현
        blur_img = convolution(pyramid_img[-1], gaussian_filter_mask) # 고주파 노이즈 제거
        downsample_img = blur_img[::downsample_num,::downsample_num] # 행,열을 2칸씩 건너 뛰면서 샘플링-->0.5배 다운샘플링
        pyramid_img.append(downsample_img)
    return pyramid_img # 다운샘플된 이미지 리스트 반환



In [ ]:
# 랜덤 기반 영역 추출
def random_region_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)
    output = np.zeros_like(img)
    h, w = img.shape
    np.random.seed(42)
    for _ in range(num_patches):
        # 패치가 이미지 경계를 넘지 않도록 좌상단 좌표 무작위로 선정
        x = np.random.randint(0, h - patch_size)
        y = np.random.randint(0, w - patch_size)
        # 나머지 영역은 모두 0(검정색)
        output[x:x+patch_size, y:y+patch_size] = img[x:x+patch_size, y:y+patch_size]

    return output
'''
# 각 픽셀을 scale_x, scale_y만큼 반복해서 엄샘플링---최근접 보간법과 동일
#upsampled = img.repeat(scale_x, axis=0).repeat(scale_y, axis=1)
#interpolated_img = NN_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = b_spline_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bilinear_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
'''

# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512)) # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512)) # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합. 높은 해상도에 더 큰 가중치
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255) # 결합된 이미지 픽셀값 0~255로 제한

#===================코드 실행 영역 ==============================
gaus_pyramid_img= gaussian_pyramid(gray_lena_img, 0.5, 3) # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]
img_256 = gaus_pyramid_img[1]
img_128 = gaus_pyramid_img[2]

# k값을 변경해가면서 추출 k:1,2,4,8,16,32,64,128
img_256_sparse = random_region_selection(img_256,16)
img_512_sparse = random_region_selection(img_512,16)

# Reconstruct using Gaussian pyramid idea
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(gray_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(10,5))
plt.subplot(1, 3, 1); plt.title("Original img"); plt.imshow(gray_lena_img, cmap='gray')
plt.subplot(1, 3, 2); plt.title('img_512_sparse'); plt.imshow(img_512_sparse, cmap='gray')
plt.subplot(1, 3, 3); plt.title('img_256_sparse'); plt.imshow(img_256_sparse, cmap='gray')
plt.show()

# Optional img vs Reconstructed img
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(gray_lena_img, cmap='gray')
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed, cmap='gray')
plt.show()


In [ ]:
# 분산 기반 영역 추출
def variance_based_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)
    h, w = img.shape
    output = np.zeros_like(img)
    variances = []

    # 모든 가능한 패치의 분산 계산
    for i in range(0, h - patch_size, patch_size):
        for j in range(0, w - patch_size, patch_size):
            patch = img[i:i+patch_size, j:j+patch_size]
            var = np.var(patch) # 패치의 분산 계산
            variances.append((var, i, j)) # 분산과 패치의 좌표를 튜플로 묶어서 저장

    variances.sort(reverse=True) # 분산이 높은 순서로 정렬--더 많은 디테일한 정보 포함
    selected_patch = variances[:num_patches] # num_patch만큼 패치수 추출
    for _,i,j in selected_patch:
    # 선택된 패치를 결과 이미지에 복사
        output[i:i+patch_size, j:j+patch_size] = img[i:i+patch_size, j:j+patch_size]

    return output


#interpolated_img = NN_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bilinear_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = b_spline_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512

# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512)) # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512)) # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합. 높은 해상도에 더 큰 가중치
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255) # 결합된 이미지 픽셀값 0~255로 제한

#===================코드 실행 영역 ==============================
gaus_pyramid_img= gaussian_pyramid(gray_lena_img, 0.5, 3) # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]
img_256 = gaus_pyramid_img[1]
img_128 = gaus_pyramid_img[2]

# k값을 변경해가면서 추출 k:1,2,4,8,16,32,64,128
img_256_sparse = variance_based_selection(img_256,16)
img_512_sparse = variance_based_selection(img_512,16)

# Reconstruct using Gaussian pyramid idea
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(gray_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
plt.imshow(gray_lena_img, cmap='gray')
plt.title('Original img')
plt.subplot(1,3,2)
plt.imshow(img_512_sparse, cmap='gray')
plt.title('img_512_sparse')
plt.subplot(1,3,3)
plt.imshow(img_256_sparse, cmap='gray')
plt.title('img_256_sparse')
plt.show()

# Optional img vs Reconstructed img
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(gray_lena_img, cmap='gray')
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed, cmap='gray')
plt.show()


In [ ]:
# 그래디언트 크기 기반 영역 추출
# sobel 필터를 사용하여 에지가 두드러진 영역 추출
# 노이즈에 민감, 부드러운 영역 무시됨, 윤곽선 보존에 유리

def sobel_filter(img): # 소벨 필터로 그래디언트 계산
    img_h, img_w = img.shape
    sobel_x_filter = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_y_filter = np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]])

    sobel_x_img = np.zeros((img.shape[0],img.shape[1])) # x축 방향 검출에지를 담을 배열
    sobel_y_img = np.zeros((img.shape[0],img.shape[1])) # y축 방향 검출에지를 담을 배열

    filter_half = sobel_x_filter.shape[0] // 2# 컨볼루션 시작 범위 인덱스

    for y in range(filter_half, img_h - filter_half):
        for x in range(filter_half, img_h - filter_half):
            img_patch = img[x - filter_half : x + filter_half + 1, y - filter_half : y + filter_half + 1]

            sobel_x_img[x, y] = np.sum(img_patch * sobel_x_filter) # x방향 소벨 필터 진행
            sobel_y_img[x, y] = np.sum(img_patch * sobel_y_filter) # y방향 소벨 필터 진행

    total_sobel_img = np.sqrt(sobel_x_img**2, sobel_y_img**2)
    total_sobel_img = np.uint8(total_sobel_img) # np.unit8() 이용하여 정규화 진행??

    return total_sobel_img

def gradient_based_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)
    h, w = img.shape
    output = np.zeros_like(img)
    gradient_scores = []

    sobel_img = sobel_filter(img)
    # 패치별 그라디언트 크기 합계 계산
    for i in range(0, h - patch_size + 1, patch_size):
        for j in range(0, w - patch_size + 1, patch_size):
            patch_grad = sobel_img[i:i+patch_size, j:j+patch_size]
            grad_score = np.sum(patch_grad)
            gradient_scores.append((grad_score, i, j))

    # 그라디언트 크기 기준으로 정렬 후 상위 패치 선택
    gradient_scores.sort(reverse=True)
    selected = gradient_scores[:num_patches]

    # 선택된 패치 출력
    for idx, (_, i, j) in enumerate(selected):
        output[i:i+patch_size, j:j+patch_size] = img[i:i+patch_size, j:j+patch_size]

    return output

#interpolated_img = NN_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bilinear_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = b_spline_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512

# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512)) # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512)) # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합. 높은 해상도에 더 큰 가중치
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255) # 결합된 이미지 픽셀값 0~255로 제한

#===================코드 실행 영역 ==============================
gaus_pyramid_img= gaussian_pyramid(gray_lena_img , 0.5, 3) # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]
img_256 = gaus_pyramid_img[1]
img_128 = gaus_pyramid_img[2]

# k값을 변경해가면서 추출 k:1,2,4,8,16,32,64,128
img_256_sparse = gradient_based_selection(img_256,16)
img_512_sparse = gradient_based_selection(img_512,16)

# Reconstruct using Gaussian pyramid idea
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(gray_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
plt.imshow(gray_lena_img, cmap='gray')
plt.title('Original img')
plt.subplot(1,3,2)
plt.imshow(img_512_sparse, cmap='gray')
plt.title('img_512_sparse')
plt.subplot(1,3,3)
plt.imshow(img_256_sparse, cmap='gray')
plt.title('img_256_sparse')
plt.show()

# Optional img vs Reconstructed img
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(gray_lena_img, cmap='gray')
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed, cmap='gray')
plt.show()


In [ ]:
# 컬러이미지 랜덤 기반
# 랜덤 기반 영역 추출
def random_region_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)
    output = np.zeros_like(img)
    h, w = img.shape
    np.random.seed(42)
    for _ in range(num_patches):
        # 패치가 이미지 경계를 넘지 않도록 좌상단 좌표 무작위로 선정
        x = np.random.randint(0, h - patch_size)
        y = np.random.randint(0, w - patch_size)
        # 나머지 영역은 모두 0(검정색)
        output[x:x+patch_size, y:y+patch_size] = img[x:x+patch_size, y:y+patch_size]

    return output
'''
# 각 픽셀을 scale_x, scale_y만큼 반복해서 엄샘플링---최근접 보간법과 동일
#upsampled = img.repeat(scale_x, axis=0).repeat(scale_y, axis=1)
#interpolated_img = NN_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = b_spline_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#interpolated_img = bilinear_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
#reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
'''

# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512)) # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512)) # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합. 높은 해상도에 더 큰 가중치
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255) # 결합된 이미지 픽셀값 0~255로 제한

#===================코드 실행 영역 ==============================
gaus_pyramid_img= gaussian_pyramid(rgb_lena_img, 0.5, 3) # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]
img_256 = gaus_pyramid_img[1]
img_128 = gaus_pyramid_img[2]

# k값을 변경해가면서 추출 k:1,2,4,8,16,32,64,128
img_256_sparse = random_region_selection(img_256,16)
img_512_sparse = random_region_selection(img_512,16)

# Reconstruct using Gaussian pyramid idea
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)



In [ ]:
# 컬러이미지 분산 기반 추출



In [ ]:
# 컬러이미지 에지 기반

